In [45]:
import json
from pathlib import Path
from typing import List, Tuple, Dict, Any

from xueqiu_funds import XueQiuFunds

In [46]:
runtime_dir = Path("__file__").parent.joinpath("runtime")
runtime_dir.mkdir(parents=True, exist_ok=True)

FeeType = Tuple[str, List[Dict[str, Any]]]

In [47]:
table = []


def run(fd_codes_):
    table.clear()

    xqf = XueQiuFunds()

    for fd_code in fd_codes_:
        detail = xqf.get_fund_detail(fd_code)
        intro = xqf.get_fund_intro(fd_code)

        if (fd_name := intro["data"]["fd_name"]) and (
                other_rate_table := detail["data"]["fund_rates"]["other_rate_table"]
        ):
            cell = {"基金名称": fd_name}
            cell_1 = {i["name"]: float(i["value"]) for i in other_rate_table}
            cell["总计"] = sum(cell_1.values())
            cell.update(cell_1)

            table.append(cell)

            # Cache memory
            fdr = runtime_dir.joinpath(f"{fd_code}_{fd_name}")
            fdr.mkdir(parents=True, exist_ok=True)

            fdr.joinpath("fund_detail.json").write_text(
                json.dumps(detail, ensure_ascii=False), encoding="utf8"
            )
            fdr.joinpath("fund_intro.json").write_text(
                json.dumps(intro, ensure_ascii=False), encoding="utf8"
            )


In [48]:
import pandas as pd

fd_codes = [
    "012832",
    "003766",
    "018735",
    "007339",
    "011613",
    "004855",
    "016858",
]

run(fd_codes)
pd.DataFrame.from_dict(table).sort_values(by="总计", ignore_index=True)

,基金名称,总计,基金管理费,基金托管费,销售服务费
0,广发创业板ETF联接C,0.40,0.15,0.05,0.20
1,易方达沪深300ETF联接C,0.40,0.15,0.05,0.20
2,南方中证新能源ETF联接C,0.50,0.15,0.05,0.30
3,广发中证全指汽车指数C,0.80,0.50,0.10,0.20
4,华夏中证绿色电力50ETF发起式联接C,0.85,0.50,0.10,0.25
5,华夏上证科创板50成份ETF联接C,0.85,0.50,0.10,0.25
6,国金量化多因子C,1.80,1.20,0.20,0.40
